In [1]:
import seagul.envs
import ray
import ray.rllib.agents.ppo as ppo
from ray.tune.logger import pretty_print
import tensorflow as tf
import time
from tqdm import trange

from ray.tune.registry import register_env
from seagul.envs.mujoco.five_link import FiveLinkWalkerEnv

env_name = "Walker2d-v3"


### Needed for custom envs only 
def env_creator(env_config):
    return FiveLinkWalkerEnv()  # return an env instance

register_env("five_link-v3", env_creator)
####


ray.init()
config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 1
config["num_workers"] = 8
config["eager"] = False
config["model"]["fcnet_hiddens"] = []

trainer = ppo.PPOTrainer(config=config, env=env_name)

# Can optionally call trainer.restore(path) to load a checkpoint.

start = time.time()
for i in trange(2000):
   # Perform one iteration of training the policy with ARS
    result = trainer.train()

checkpoint = trainer.save()
print("checkpoint saved at", checkpoint)
print(pretty_print(result))
ray.shutdown()

/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sgillen/miniconda3/envs/ray/lib/python3.6/s

(pid=23993) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=23993)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=23993) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=23993)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=23993) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=23993)   _np_qint16 = np.dtype([("q

2019-11-05 12:05:46,471	WARNING util.py:45 -- Install gputil for GPU system monitoring.
  0%|          | 0/2000 [00:00<?, ?it/s]

(pid=23984) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(pid=23984)   "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
(pid=23993) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(pid=23993)   "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
(pid=23988) /home/sgillen/miniconda3/envs/ray/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(pid=23988)   "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
(pid=23995) /hom

2019-11-05 12:05:48,835	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_out/kernel:0' shape=(17, 12) dtype=float32>
2019-11-05 12:05:48,836	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/fc_out/bias:0' shape=(12,) dtype=float32>
2019-11-05 12:05:48,836	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/value_out/kernel:0' shape=(17, 1) dtype=float32>
2019-11-05 12:05:48,837	INFO tf_policy.py:358 -- Optimizing variable <tf.Variable 'default_policy/value_out/bias:0' shape=(1,) dtype=float32>
2019-11-05 12:05:48,856	INFO multi_gpu_impl.py:146 -- Training on concatenated sample batches:

{ 'inputs': [ np.ndarray((4000, 6), dtype=float32, min=-3.925, max=4.118, mean=0.006),
              np.ndarray((4000,), dtype=float32, min=-2.229, max=2.205, mean=-0.013),
              np.ndarray((4000, 17), dtype=float32, min=-10.0, max=10.0, mean=-0.993),
              np.ndarray((4000,), dtype=float32, min=-17.084, max=-5.6

checkpoint saved at /home/sgillen/ray_results/PPO_Walker2d-v3_2019-11-05_12-05-42omv0tj3y/checkpoint_2000/checkpoint-2000
custom_metrics: {}
date: 2019-11-05_13-21-42
done: false
episode_len_mean: 183.22
episode_reward_max: 494.1734154149198
episode_reward_mean: 321.92980580863616
episode_reward_min: 169.6333219093705
episodes_this_iter: 22
episodes_total: 45595
experiment_id: 640cac2ff58d462c9c3922b66afaa257
hostname: sgdt
info:
  grad_time_ms: 1401.347
  learner:
    default_policy:
      cur_kl_coeff: 0.6164836287498474
      cur_lr: 4.999999873689376e-05
      entropy: 11.495804786682129
      entropy_coeff: 0.0
      kl: 0.00945001095533371
      policy_loss: -0.018961669877171516
      total_loss: 296.18170166015625
      vf_explained_var: 0.7680788636207581
      vf_loss: 296.19488525390625
  load_time_ms: 2.508
  num_steps_sampled: 8000000
  num_steps_trained: 7936000
  sample_time_ms: 998.629
  update_time_ms: 4.039
iterations_since_restore: 2000
node_ip: 192.168.0.12
num_heal

In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt

env = gym.make(env_name)
obs = env.reset()

action_hist = []
state_hist  = []
reward_hist = []

done = False
while not done:
    actions = trainer.compute_action(obs.flatten())
    obs, reward, done, _ = env.step(np.asarray(actions))
    action_hist.append(np.copy(np.clip(actions, -10, 10)))
    state_hist.append(np.copy(obs))
    reward_hist.append(np.copy(reward))
    if done:
        break


print(sum(reward_hist))
plt.plot(action_hist)
plt.figure()
plt.plot(state_hist)

2019-11-05 14:32:29,688	INFO tf_run_builder.py:92 -- Executing TF run without tracing. To dump TF timeline traces to disk, set the TF_TIMELINE_DIR environment variable.


319.22329804052384
